In [3]:
import carla
import time

client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()

def spawn_vehicle(vehicle_index=0, spawn_index=0, x_offset=0, y_offset=0, pattern='vehicle.*model3*') -> carla.Vehicle:
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    spawn_point.location.x += x_offset
    spawn_point.location.y += y_offset
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

# Spawn target vehicle
target_vehicle = spawn_vehicle(x_offset=-50)

time.sleep(5)

# Spawn the vehicle
ego_vehicle = spawn_vehicle()

# Add the radar sensor
radar_bp = world.get_blueprint_library().find('sensor.other.radar')
radar_bp.set_attribute('horizontal_fov', '10')  # Horizontal field of view
radar_bp.set_attribute('vertical_fov', '10')    # Vertical field of view
radar_bp.set_attribute('range', '20')           # Maximum range

radar_transform = carla.Transform(carla.Location(x=2.0, z=1.0))
radar = world.spawn_actor(radar_bp, radar_transform, attach_to=ego_vehicle)

# Variable to store the minimum TTC
min_ttc = float('inf')

def radar_callback(data: carla.RadarMeasurement):
    global min_ttc, min_distance
    min_ttc = float('inf')

    for detection, i in zip(data, range(len(data))):
        absolute_speed = abs(detection.velocity)

        # Calculate TTC
        if absolute_speed != 0:
            ttc = detection.depth / absolute_speed
            print(ttc)
            if ttc < min_ttc:
                min_ttc = ttc

# Register the radar callback
radar.listen(radar_callback)
i=0
try:
    while True:
        # TTC threshold (e.g., 2 seconds)
        ttc_threshold = 1.0

        # Braking control
        if min_ttc < ttc_threshold:
            control = carla.VehicleControl()
            control.brake = 1.0  # Maximum braking
            ego_vehicle.apply_control(control)
            print("Emergency braking activated!")
            i += 1
            if i>10:
                break
        else:
            control = carla.VehicleControl()
            control.throttle = 0.5  # Maintain constant speed
            ego_vehicle.apply_control(control)
        
        time.sleep(0.05)
    time.sleep(2)
except KeyboardInterrupt:
    print("Keyboard interrupt detected.")
finally:
    radar.stop()
    radar.destroy()
    ego_vehicle.destroy()
    target_vehicle.destroy()


0.0009949474041488636
393.63767229138415
172.6070765198847
205.44767236698033
250.77245152887
201.7417801439823
211.34594575183846
139.7181485856429
127.56382849888462
193.5527250157812
124.08761286805351
119.6508357188486
188.68529212875353
178.7097882353716
85.15317014798642
185.88229673326654
142.79091985123225
195.12637038662808
99.94514007493413
232.1776266858217
217.4228109173551
309.19985364976105
415.54739870360896
361.23294251535145
384.14549430350155
224.03951231814364
923.6992601701667
477.0020131078086
1082.5591007085413
887.0862626056946
1205.2939352852534
1722.4707569587656
916.4308507402137
1723.1478553213697
2232.188046938272
3492.303523038202
1475.0015225142217
3405.8190364087427
13144.146754370615
9066.98970564498
50434.86592097609
45169.981115139606
39148.93793557068
18957.012483585993
5911.497558439118
6915.757461919136
3374.540176867076
2497.6855664480913
2305.9931029193012
2909.043102254519
2419.6076553391636
2200.3154235701477
1474.168343864753
1429.3076037800147